## In-class Lesson 2: Pivots, Joins and Functions in `pandas`

Building off last week's lesson, we'll be working with some election results data today as we learn how to use the `pivot_table` and `merge` methods in `pandas`.

We'll be working with two csvs today. The first called `ks_ballot.csv` is a file of county-level results from the Kansas Constitutional Amendment on abortion access. The second called `ks_prez.csv` is a file containing county-level presidential results in Kansas from the 2020 election.

Our goal: Join the two csvs together into a single dataframe and compare the 2020 presidential results to the results of the ballot initiative.

## Import `pandas`

In [1]:
import pandas as pd

## Load `ks_ballot.csv` from your computer

`pandas` locates files using their location on your computer, known as the path. Paths look like this for example:

```/Users/user/Downloads/file.csv```

Below we can use the `./data/` because the files are in the `data` folder with our notebook.

In [2]:
ks_ballot = pd.read_csv('./data/ks_ballot.csv')

ks_ballot.head()

,fips,choice,votes
0,20001,No,1816
1,20001,Yes,1836
2,20003,No,950
3,20003,Yes,1418
4,20005,No,2449


## Inspecting our data

You'll notice a few familiar things about this data. One, it contains a county level `fips` code. But notice that it is different than other county-level data we've worked with. Getting a unique count of the `fips` column using pandas `value_counts()` method will illustrate the difference.

In [4]:
ks_ballot['fips'].value_counts()

20209    2
20103    2
20055    2
20057    2
20059    2
        ..
20145    2
20147    2
20149    2
20151    2
20001    2
Name: fips, Length: 105, dtype: int64

As we see, each county fips is repeated twice because there are two answers on the ballot measure: Yes or No. This data is stored in what is called `long format` where each row is a separate entry for each ballot choice. Our goal is to join this data to the presidential results data, so let's take a look at it.

## Load `ks_prez.csv` from our computer

In [5]:
ks_prez = pd.read_csv('./data/ks_prez.csv')

ks_prez.head()

,fips,county_name,state_code,acp_type,biden,trump,other,prez_total
0,20001,Allen County,KS,Rural Middle America,1570,4218,104,5892
1,20003,Anderson County,KS,Rural Middle America,782,2929,81,3792
2,20005,Atchison County,KS,Rural Middle America,2359,4906,175,7440
3,20007,Barber County,KS,Rural Middle America,291,2014,37,2342
4,20009,Barton County,KS,Rural Middle America,2340,8608,182,11130


Notice here our data is stored differently. Not only do we have additional details about the counties (name, state and a value called `acp_type` but it also appears to be stored in what is called `wide format` where each county has it's own row. To be sure, let's use `value_counts()` again on the `fips` column.


In [7]:
ks_prez['fips'].value_counts()

20209    1
20103    1
20055    1
20057    1
20059    1
        ..
20145    1
20147    1
20149    1
20151    1
20001    1
Name: fips, Length: 105, dtype: int64

As expected, each fips appears only once in the presidential election data. Since our goal is to join these two dataframes together, we are going to need to convert them to the same format. For this exercise, we'll take the long formatted ballot initiative results and turn them into wide format, which will make for an easier comparison of the two elections in each county. To do this, we will use a tool we've used before: a PivotTable. 

In `pandas`, creating a PivotTable is very similar to how we did it in Excel or GoogleSheets. We choose the rows, or `index` in pandas, the columns and the values to aggregate, usually a `count` or `sum`. 

Returning to `ks_ballot`, our index will be the `fips` code because we want to group our data into a table that looks something like this:

| fips  | yes  | no   |
|-------|------|------|
| 20001 | 1836 | 1816 |
| 20003 | 1418 | 950  |

## Working with fips and other numbers stored as text

Because we're working with `fips` codes, I want to pause here and reimport our data passing an argument that will save us some headaches later. You'll probably recall that some `fips` codes have a leading zero, such as `05001`, which can be cut off if the code is read is as a number, rather than as text. We don't have that issue with Kansas because it's fips code is `20` but that may not be the case every time. To make sure you don't lose that zero, we'll use the `dtype` argument in the `read_csv` method to set the `fips` column as text, or `object` in `pandas`, right from the start.  

In [8]:
ks_prez = pd.read_csv('./data/ks_prez.csv', dtype={'fips':'object'})

ks_ballot = pd.read_csv('./data/ks_ballot.csv', dtype={'fips':'object'})

## Inspecting datatypes

Let's take a look at the datatypes for each dataframe using the `dtypes` method in `pandas`. And we'll see the `fips` column in each is now stored as an object.

In [9]:
ks_ballot.dtypes

fips      object
choice    object
votes      int64
dtype: object

In [10]:
ks_prez.dtypes

fips           object
county_name    object
state_code     object
acp_type       object
biden           int64
trump           int64
other           int64
prez_total      int64
dtype: object

## Creating a `pivot_table` in pandas

There are a few different ways to pivot and group data in `pandas`. We'll be using the `pivot_table` method in this example. You can read the full documentation [here](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html).

First, let's look at the columns we have in our dataframe. You can returns these as a list like below.

## List columns in a dataframe

In [11]:
ks_ballot.columns.to_list()

['fips', 'choice', 'votes']

Let's look at the code below. First, we invoke the `pivot_table` method from pandas, which we've assigned the variable name `pd`. Next, we'll pass lists to the method using our column names. 

We'll be summing vote totals, so `votes` goes in values.

We want to group by the `fips`, so we'll pass `fips` to the index argument.

We want to break out our `choice` column into two seperate columns becase on the unique values in the columns. So we pass it to the columns argument.

Lastly, I'm going to do two things to our pivot_table. I'm going to reset the index so that `fips` becomes a true column in our destination dataframe. I'm also going to fill any null values with 0's becauses we'll be doing math on this data.

Let's try it out, assigning the pivot table as a variable named `ks_wide`

In [14]:
ks_wide = pd.pivot_table(
                ks_ballot, 
               values= ['votes'], 
               index = ['fips'], 
               columns=['choice'], 
               aggfunc=sum
            ).reset_index(
            ).fillna(0)

ks_wide.head()

fips votes      
choice           No   Yes
0       20001  1816  1836
1       20003   950  1418
2       20005  2449  2472
3       20007   458   998
4       20009  3275  4022

This looks like what we want with the exception of the names of the columns. We have created a dataframe that has two layers of column headers when we really only want one layer. We can fix that by renaming the columns to names of our choice using a list.

## Rename columns

In [15]:
ks_wide.columns = ['fips', 'no', 'yes']

ks_wide.head()

,fips,no,yes
0,20001,1816,1836
1,20003,950,1418
2,20005,2449,2472
3,20007,458,998
4,20009,3275,4022


Now we have what we want. A single row per county with no votes and yes votes in their own columns. Now, let's do a little math. 

Because we're going to be comparing this to vote percentages, let's calculate a total of votes in one column and then calculate the vote shares for yes and no. Let's also round those vote shares to three decimal places and multiply by 100 to represent them as percentages. We can do this pretty easily in pandas like below.

In [16]:
ks_wide['total'] = ks_wide['no'] + ks_wide['yes']

ks_wide['yes_pct'] = round(ks_wide['yes'] / ks_wide['total'], 3) * 100

ks_wide['no_pct'] = round(ks_wide['no'] / ks_wide['total'], 3) * 100

ks_wide.head()

,fips,no,yes,total,yes_pct,no_pct
0,20001,1816,1836,3652,50.3,49.7
1,20003,950,1418,2368,59.9,40.1
2,20005,2449,2472,4921,50.2,49.8
3,20007,458,998,1456,68.5,31.5
4,20009,3275,4022,7297,55.1,44.9


Let's also add percentages to our presidential results data so we can compare the vote shares later when we join the tables together.

In [17]:
ks_prez

,fips,county_name,state_code,acp_type,biden,trump,other,prez_total
0,20001,Allen County,KS,Rural Middle America,1570,4218,104,5892
1,20003,Anderson County,KS,Rural Middle America,782,2929,81,3792
2,20005,Atchison County,KS,Rural Middle America,2359,4906,175,7440
3,20007,Barber County,KS,Rural Middle America,291,2014,37,2342
4,20009,Barton County,KS,Rural Middle America,2340,8608,182,11130
...,...,...,...,...,...,...,...,...
100,20201,Washington County,KS,Aging Farmlands,475,2363,45,2883
101,20203,Wichita County,KS,Hispanic Centers,149,808,11,968
102,20205,Wilson County,KS,Working Class Country,723,3153,78,3954
103,20207,Woodson County,KS,Working Class Country,294,1228,24,1546


In [19]:
ks_prez['biden_pct'] = round(ks_prez['biden'] / ks_prez['prez_total'], 3) * 100

ks_prez['trump_pct'] = round(ks_prez['trump'] / ks_prez['prez_total'], 3) * 100

ks_prez

,fips,county_name,state_code,acp_type,biden,trump,other,prez_total,biden_pct,trump_pct
0,20001,Allen County,KS,Rural Middle America,1570,4218,104,5892,26.6,71.6
1,20003,Anderson County,KS,Rural Middle America,782,2929,81,3792,20.6,77.2
2,20005,Atchison County,KS,Rural Middle America,2359,4906,175,7440,31.7,65.9
3,20007,Barber County,KS,Rural Middle America,291,2014,37,2342,12.4,86.0
4,20009,Barton County,KS,Rural Middle America,2340,8608,182,11130,21.0,77.3
...,...,...,...,...,...,...,...,...,...,...
100,20201,Washington County,KS,Aging Farmlands,475,2363,45,2883,16.5,82.0
101,20203,Wichita County,KS,Hispanic Centers,149,808,11,968,15.4,83.5
102,20205,Wilson County,KS,Working Class Country,723,3153,78,3954,18.3,79.7
103,20207,Woodson County,KS,Working Class Country,294,1228,24,1546,19.0,79.4


In [20]:
ks_prez.head()

,fips,county_name,state_code,acp_type,biden,trump,other,prez_total,biden_pct,trump_pct
0,20001,Allen County,KS,Rural Middle America,1570,4218,104,5892,26.6,71.6
1,20003,Anderson County,KS,Rural Middle America,782,2929,81,3792,20.6,77.2
2,20005,Atchison County,KS,Rural Middle America,2359,4906,175,7440,31.7,65.9
3,20007,Barber County,KS,Rural Middle America,291,2014,37,2342,12.4,86.0
4,20009,Barton County,KS,Rural Middle America,2340,8608,182,11130,21.0,77.3


In [21]:
ks_wide.head()

,fips,no,yes,total,yes_pct,no_pct
0,20001,1816,1836,3652,50.3,49.7
1,20003,950,1418,2368,59.9,40.1
2,20005,2449,2472,4921,50.2,49.8
3,20007,458,998,1456,68.5,31.5
4,20009,3275,4022,7297,55.1,44.9


## Joining dataframes

Now that we have each dataframe formated as we'd like. We're ready to join them together. In `pandas`, there are a few ways we can do this but today we'll be using the `merge` method.

Just like joining in SQL, we will use a column containig common values in each data frame. In this case, that's the `fips` column in each dataframe. We know these dataframes each contain a record per county so we'll be using an `inner` join to connect records that appear in both dataframes. 

Also like SQL, `merge` uses a left/right frame of reference. So the first table we pass in is the left table and the second table is the right table. In our example, it doesn't really matter which one is which, particularly since we can always rearrange the columns later.

Below we pass in `ks_prez` and `ks_wide` and then assign the joined dataframe to the variable `ks`.

In [22]:
ks = pd.merge(ks_prez, ks_wide, how='inner', on='fips')
ks

,fips,county_name,state_code,acp_type,biden,trump,other,prez_total,biden_pct,trump_pct,no,yes,total,yes_pct,no_pct
0,20001,Allen County,KS,Rural Middle America,1570,4218,104,5892,26.6,71.6,1816,1836,3652,50.3,49.7
1,20003,Anderson County,KS,Rural Middle America,782,2929,81,3792,20.6,77.2,950,1418,2368,59.9,40.1
2,20005,Atchison County,KS,Rural Middle America,2359,4906,175,7440,31.7,65.9,2449,2472,4921,50.2,49.8
3,20007,Barber County,KS,Rural Middle America,291,2014,37,2342,12.4,86.0,458,998,1456,68.5,31.5
4,20009,Barton County,KS,Rural Middle America,2340,8608,182,11130,21.0,77.3,3275,4022,7297,55.1,44.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,20201,Washington County,KS,Aging Farmlands,475,2363,45,2883,16.5,82.0,510,1453,1963,74.0,26.0
101,20203,Wichita County,KS,Hispanic Centers,149,808,11,968,15.4,83.5,147,534,681,78.4,21.6
102,20205,Wilson County,KS,Working Class Country,723,3153,78,3954,18.3,79.7,1015,1310,2325,56.3,43.7
103,20207,Woodson County,KS,Working Class Country,294,1228,24,1546,19.0,79.4,404,603,1007,59.9,40.1


Congrats! You've joined two dataframes in `pandas`. We now have one table with the presidential results and the ballot initiative choices.

We are ready to do some analysis. Let's start with just a few questions:

## What were the statewide totals for `Yes` and `No`?

We can get this by using pandas `sum` method.

In [23]:
ks[['yes','no','total']].sum()

yes      374611
no       534134
total    908745
dtype: int64

## How many counties did `No` win? Which ones?

Because there are only two options, Yes or No, we can answer this by looking purely at the vote share percentages above and below 50 percent. As we've discussed before, when there are more candidates, we need to use a different method finding the largest vote getter of a group to assign a winner (candidates can and often do win races with less than 50% of the vote). 

We can do this with a filter and using `1en()` to get the count.

In [24]:
no_cos = ks[ks['no_pct'] > 50]

no_cos

,fips,county_name,state_code,acp_type,biden,trump,other,prez_total,biden_pct,trump_pct,no,yes,total,yes_pct,no_pct
17,20035,Cowley County,KS,Evangelical Hubs,4273,9656,302,14231,30.0,67.9,4633,4240,8873,47.8,52.2
18,20037,Crawford County,KS,College Towns,6179,10045,421,16645,37.1,60.3,5582,4520,10102,44.7,55.3
22,20045,Douglas County,KS,College Towns,40785,17286,1424,59495,68.6,29.1,36549,8461,45010,18.8,81.2
29,20059,Franklin County,KS,Rural Middle America,3690,8479,308,12477,29.6,68.0,4863,3836,8699,44.1,55.9
30,20061,Geary County,KS,Military Posts,3983,5323,297,9603,41.5,55.4,3132,1984,5116,38.8,61.2
36,20073,Greenwood County,KS,Working Class Country,569,2444,64,3077,18.5,79.4,963,957,1920,49.8,50.2
39,20079,Harvey County,KS,Rural Middle America,6747,10182,380,17309,39.0,58.8,6381,5667,12048,47.0,53.0
42,20085,Jackson County,KS,Rural Middle America,1881,4517,186,6584,28.6,68.6,2338,2150,4488,47.9,52.1
43,20087,Jefferson County,KS,Rural Middle America,3194,6334,254,9782,32.7,64.8,3643,2946,6589,44.7,55.3
45,20091,Johnson County,KS,Exurbs,184259,155631,7324,347214,53.1,44.8,166060,76767,242827,31.6,68.4


In [25]:
len(no_cos)

19

## How many `No` counties were also Trump counties in 2020? Which ones?

This adds another level of complexity to our analysis.

In [26]:
trump_nos = no_cos[no_cos['trump_pct'] > no_cos['biden_pct']]

trump_nos

,fips,county_name,state_code,acp_type,biden,trump,other,prez_total,biden_pct,trump_pct,no,yes,total,yes_pct,no_pct
17,20035,Cowley County,KS,Evangelical Hubs,4273,9656,302,14231,30.0,67.9,4633,4240,8873,47.8,52.2
18,20037,Crawford County,KS,College Towns,6179,10045,421,16645,37.1,60.3,5582,4520,10102,44.7,55.3
29,20059,Franklin County,KS,Rural Middle America,3690,8479,308,12477,29.6,68.0,4863,3836,8699,44.1,55.9
30,20061,Geary County,KS,Military Posts,3983,5323,297,9603,41.5,55.4,3132,1984,5116,38.8,61.2
36,20073,Greenwood County,KS,Working Class Country,569,2444,64,3077,18.5,79.4,963,957,1920,49.8,50.2
39,20079,Harvey County,KS,Rural Middle America,6747,10182,380,17309,39.0,58.8,6381,5667,12048,47.0,53.0
42,20085,Jackson County,KS,Rural Middle America,1881,4517,186,6584,28.6,68.6,2338,2150,4488,47.9,52.1
43,20087,Jefferson County,KS,Rural Middle America,3194,6334,254,9782,32.7,64.8,3643,2946,6589,44.7,55.3
51,20103,Leavenworth County,KS,Military Posts,13886,21610,994,36490,38.1,59.2,14385,9867,24252,40.7,59.3
55,20111,Lyon County,KS,College Towns,6055,7550,383,13988,43.3,54.0,6037,3562,9599,37.1,62.9


In [27]:
len(trump_nos)

14

## In how many, if any counties, did `yes` get a higher vote share than former President Donald Trump? Which counties?

In [28]:
ks[ks['yes_pct'] > ks['trump_pct']]

,fips,county_name,state_code,acp_type,biden,trump,other,prez_total,biden_pct,trump_pct,no,yes,total,yes_pct,no_pct


You can add even more questions and analysis from here if you like. Hopefully this has helped you start to feel comfortable with some of the ways `pandas` can help with your data analysis.

## Lastly, let's save `ks` to a csv in case we want to use it elsewhere.

In [29]:
ks.to_csv('./data/ks.csv', index=False)